In [ ]:
### COMPUTATIONAL PHYSICS
### Homework 2, Problem 1: Integration

# In physics calculations we often encounter functions which do not have a nice 
# closed form expression. One of these is the gamma function.

# Γ(a) = int( x**(a-1) * e**(-x) ) from 0 to inf.

# For this problem feel free to use scipy integrate where appropriate, 
# but you must explicitly choose your integrator and not let scipy choose for you.

#a) Write a program to make a graph of the value of the integral given for the 
#.  bounds x=0-5 and for a = 2. On the same axis plot the result for ours five 
#.  integration methods (Trapezoid, Gaussian Quadrature, Simpson’s, Romberg and 
#.  Monte Carlo. On two other sets of axis show the value as a function of 
#.  N=8,16,32,63,128,256 for each method and plot the estimated error as a function of N.

#b) Choose what you think is the best integrator for this method and the N required 
#.  for accuracy. Justify your choices and write a program to make a graph of the 
#.  value of the integral given from x=0-5 for a = 2,3, and 4 on the same axis.

#c) In order to numerically integrate x = 0 − ∞ we need to change variables. Since 
#.  most of value of the integral is near the peak, what is the change of variables 
#.  which shifts the range of integration from 0 to 1 and places the peak near 1/2.

#d) We next need to transform the integral to make it a bit easier to evaluate. This 
#.  can be done by writing xa−1 as e(a−1)lnx. Using this new transformation calculate 
#.  to write a user defined function Γ(a) which will calculate Γ for an arbitrary a value. 
#.  Test your function with the known value of Γ(1.5) = 0.5􏰁(π).

#e) Plot the Gamma function for a = 1-10.

#f) Show that for integer values of a the Gamma function is equal to the factorial of a. 
#.  (HINT: Write a user defined function to calculate the factorial of an integer).

### Name:  Carson Huey-You

In [ ]:
# Import Functions

import math as math
import numpy as np
from matplotlib import pyplot as plt
import scipy.integrate as scipy_int

In [ ]:
# Define function as the integrand of Γ(a)

def func(x, a):
    y = x**(a-1) * np.e**(-x)
    return y

func_vec = np.vectorize(func, excluded='a')
func_vec_a = np.vectorize(func, excluded='x')

In [ ]:
# Define function for Monte-Carlo integration

def montecarlo_int(xmin, xmax, a, N):
    
    N_mc = N*10000
    x = np.linspace(xmin, xmax, N)
    
    ymax = np.amax(func(x, a)) + 0.25
    ymin = 0
    
    x_mc = np.random.random(N_mc)*(xmax - xmin) + xmin
    y_mc = np.random.random(N_mc)*(ymax - ymin) + ymin
    
    mask = ( y_mc <= func_vec(x_mc, a) )
    count = len( y_mc[mask] )
        
    #A is the area of the 'box'
    A = (xmax - xmin)*(ymax - ymin)
    
    I = count*A/N_mc
    return I


In [ ]:
# Define function to integrate the gamma function multiple ways:

def integrate(xmin, xmax, a, N, show):
    
    x = np.linspace(xmin, xmax, N)
    
    int_trapz = scipy_int.trapz(func(x, a), x=x)
    int_gaussquad, err = scipy_int.quadrature(func_vec, xmin, xmax, args=(a))
    int_simpsons = scipy_int.simps(func(x, a), x=x)
    int_romberg = scipy_int.romberg(func_vec, xmin, xmax, [a])
    int_montecarlo = montecarlo_int(xmin, xmax, a, N)
    
    if show == True:
        state = f"For a = {a}:\n"
        trapz_state = f"Trapezoid Integration found {int_trapz} with step size of {N}.\n"
        gaussquad_state = f"Gaussian Quadrature Integration found {int_gaussquad}.\n"
        simpsons_state = f"Simpsons Integration found {int_simpsons} with step size of {N}.\n"
        romberg_state = f"Romberg Integration found {int_romberg}.\n"
        montecarlo_state = f"Monte Carlo Integration found {int_montecarlo} with {N*10000} points.\n"
        
        print(state + trapz_state + gaussquad_state + simpsons_state + romberg_state + montecarlo_state)
    
    return [int_trapz, int_gaussquad, int_simpsons, int_romberg, int_montecarlo, err]


In [ ]:
# Define change-of-vars function:

def cov_func(u, a):
    (-1)**a * ( np.ln(u) )**(a-1)
    return y

func_vec = np.vectorize(func, excluded='a')

In [ ]:
xmin = 0
xmax = 5

N = 1001

x = np.linspace(xmin, xmax, N)

In [ ]:
a = 2
plt.plot(x, func_vec(x, a))

In [ ]:
#a1) Test Integrations for multiple types.

print(integrate(0, 5, 2, 100, True))

In [ ]:
#a2) Plot different values of N.

N = [8, 16, 32, 64, 128, 256]

trapz_arr = np.zeros_like(N, dtype='float64')
gaussquad_arr = np.zeros_like(N, dtype='float64')
simpsons_arr = np.zeros_like(N, dtype='float64')
romberg_arr = np.zeros_like(N, dtype='float64')
montecarlo_arr = np.zeros_like(N, dtype='float64')
gaussquad_err_arr = np.zeros_like(N, dtype='float64')

for k in range(0, len(N)):
    
    int_result_k = integrate(0, 5, 2, N[k], False)
    
    trapz_arr[k] = int_result_k[0]
    gaussquad_arr[k] = int_result_k[1]
    simpsons_arr[k] = int_result_k[2]
    romberg_arr[k] = int_result_k[3]
    montecarlo_arr[k] = int_result_k[4]
    gaussquad_err_arr[k] = int_result_k[5]
    
    print(f"Completed integration for N = {N[k]}.")
    

In [ ]:
#a3) Plot Figure... Note GaussQuad and Romberg don't depend on N.

fig = plt.figure(figsize=(20, 10))
fig.subplots_adjust(hspace=0.0, wspace=0.0)

plt.title('Integration Types for the Gamma Function\n', size=15)

ax = fig.add_subplot(1, 1, 1)
ax.set_xticks([])
ax.set_yticks([])

ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(N, trapz_arr, label = f"Trapezoidal")
#ax1.plot(N, gaussquad_arr, label = f"GaussQuad")
ax1.plot(N, simpsons_arr, label = f"Simpsons")
#ax1.plot(N, romberg_arr, label = f"Romberg")
ax1.plot(N, montecarlo_arr, label = f"Monte Carlo")

ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(N, gaussquad_err_arr, label = f"Error from GaussQuad")

ax1.legend()
ax2.legend()


In [ ]:
#b) Choosing Gaussian Quadrature, as it converges *very* quickly and returns error.

x = np.linspace(0, 5, 1000)

array_a2 = func(x, 2)
quad_a2, err_a2 = scipy_int.quadrature(func_vec, xmin, xmax, args=(2))

array_a3 = func(x, 3)
quad_a3, err_a3 = scipy_int.quadrature(func_vec, xmin, xmax, args=(3))

array_a4 = func(x, 4)
quad_a4, err_a4 = scipy_int.quadrature(func_vec, xmin, xmax, args=(4))


fig = plt.figure(figsize=(15, 10))
fig.subplots_adjust(hspace=0.0, wspace=0.0)

plt.title('Values of the Gamma Function', size=13)

ax = fig.add_subplot(1, 1, 1)
ax.set_xticks([])
ax.set_yticks([])

ax1 = fig.add_subplot(3, 1, 1)
ax1.plot(x, array_a2, label = f"Integral for a=2: {quad_a2} $\pm$ {err_a2}")

ax2 = fig.add_subplot(3, 1, 2)
ax2.plot(x, array_a3, label = f"Integral for a=3: {quad_a3} $\pm$ {err_a3}")

ax3 = fig.add_subplot(3, 1, 3)
ax3.plot(x, array_a4, label = f"Integral for a=4: {quad_a4} $\pm$ {err_a4}")


ax1.legend()
ax2.legend()
ax3.legend(loc = 'lower right')

In [ ]:
#c) Use x = -ln(u), or u = e^(-x)

#  At x = 0  ->  u = e^(-0) = 1
#  At x = ∞  ->  u = e^(-∞) = 0

#  du/dx = -e^(-x), noting that   - du = dx * e^(-x)

#  The integral is then simply:

#  (-1)^a * [ln(u)]^(a-1) du, evaluated from u=0 to u=1
